---

**APRP: 3. HMM, Parameter and Structure Learning**

---


**Contents:**

--------

**PART 1: Hidden Markov Model for sequence annotation**
1. Understand sequence annotation annotation components
2. Understand HMM
3. Implementation and testing


**PART 2: Parameter and Structure Learning**
1. Short context
2. Parameter Learning in a Bayes Net
3. Structure Learning in a Bayes Net




# **PART 1: Hidden Markov Model for sequence annotation**

### 1. Short context and intro

Main links
- DNA: https://en.wikipedia.org/wiki/DNA
- Coding region: https://en.wikipedia.org/wiki/Coding_region




### 2. Setting up the enviroment
Loading common Libraries.

To consider:
-seq: sequence to analyse
-base: 4 possible bases Adenine Guanine Uracil Cytosine
-emissions: emission probability for each state
-states: possible states to annotate the sequence
-transistions: matrix with the stats prob transitions

In [1]:
import numpy as np

seq= "AGCTAGCAGTATGTCATGGCATGTTCGGAGGTAGTACGTAGAGGTAGCTAGTATAGGTCGATAGTACGCGA"

bases = "ACTG"
emissions = (0.25,0.25,0.25,0.25)
states = ("intergenic","atg","exon","gt","intron","ag","gta")
transitions =((0.99,0.01,0,0,0,0,0),
			 (0,0,1,0,0,0,0),
			 (0,0,0.9,0.05,0,0,0.05),
			 (0,0,0,0,1,0,0),
			 (0,0,0,0,0.9,0.1,0),
			 (0,0,1,0,0,0,0),
			 (1,0,0,0,0,0,0))

### 2. Callenges

1. Implement an HMM (generic)
2. Evaluate the most probable set of states for the given sequence








# **PART 2: Parameter and structure Learning**

### 1. Short context

So far we have been focused on how Bayesian networks efficiently encode a probability distribution over a set of variables.
This Assignment will be about obtaining a Bayesian network, given a set of sample data. First we focus on the problem of **parameter Learning** for a given network and secondly on **learning the structure** itself.




### 2. Parameter Learning in a Bayes Net
## 2.1. Introductory example


In [2]:
!pip install pgmpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 902.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [3]:
#Based on https://pgmpy.org/detailed_notebooks/10.%20Learning%20Bayesian%20Networks%20from%20Data.html
import numpy as np
import pandas as pd
from pgmpy.models import BayesianModel
from pgmpy.estimators import BayesianEstimator
from pgmpy.estimators import MaximumLikelihoodEstimator

# Model
data = pd.DataFrame(data={'fruit': ["banana", "apple", "banana", "apple", "banana","apple", "banana",
                                    "apple", "apple", "apple", "banana", "banana", "apple", "banana",],
                          'tasty': ["yes", "no", "yes", "yes", "yes", "yes", "yes",
                                    "yes", "yes", "yes", "yes", "no", "no", "no"],
                          'size': ["large", "large", "large", "small", "large", "large", "large",
                                    "small", "large", "large", "large", "large", "small", "small"]})
model = BayesianModel([('fruit', 'tasty'), ('size', 'tasty')]) #tasty condicionado pela fruta e pelo tamanho


# Use a MLE estimator to obtain the CPDs tables
print("MLE Estimator")
mle = MaximumLikelihoodEstimator(model, data)
print(mle.estimate_cpd('fruit'))  # unconditional
print(mle.estimate_cpd('tasty'))  # conditional

# Use a Bayesian estimator to obtain the CPDs tables
print("Bayesian Estimator")
est = BayesianEstimator(model, data)
print(est.estimate_cpd('tasty', prior_type='BDeu', equivalent_sample_size=10))

MLE Estimator
+---------------+-----+
| fruit(apple)  | 0.5 |
+---------------+-----+
| fruit(banana) | 0.5 |
+---------------+-----+
+------------+--------------+--------------------+---------------------+---------------+
| fruit      | fruit(apple) | fruit(apple)       | fruit(banana)       | fruit(banana) |
+------------+--------------+--------------------+---------------------+---------------+
| size       | size(large)  | size(small)        | size(large)         | size(small)   |
+------------+--------------+--------------------+---------------------+---------------+
| tasty(no)  | 0.25         | 0.3333333333333333 | 0.16666666666666666 | 1.0           |
+------------+--------------+--------------------+---------------------+---------------+
| tasty(yes) | 0.75         | 0.6666666666666666 | 0.8333333333333334  | 0.0           |
+------------+--------------+--------------------+---------------------+---------------+
Bayesian Estimator
+------------+---------------------+----------

## 2.2. Questions
Analyse the previous results to provide an answer the following questions:

1. For MLE, why small bananas are not tasty?
2. Why the CPDs tables for MLE and Bayesian estimators differ?
3. What is the goal of the *prior_type*?

1.

Existem 2 bananas pequenas e 1 delas é classificada como saborosa. Provavelmente o MLE estimou que as bananas pequenas não são saborosas porque no dataset o número de bananas pequenas classificadas como não saborosas é maior que o das classificadas como saborosas.

2.

As CPDs do MLE e Bayesian Estimator diferem porque o MLE assume uma distribuição a priori uniforme e pode dar overfitting aos dados, originando CPDs menos corretas. Quanto ao Bayesian Estimator, como este envolve conhecimento a priori, origina CPDs mais corretas.

3.

É um argumento utilizado para espicificar o tipo de distribuição a priori usada para estimar os parâmetros do modelo. No caso acima, o BDeu (Bayesian Dirichlet equivalent uniform) utiliza uma distribuição de Dirichlet com parâmetros uniforme a priori.


### 3. Structure Learning
## 3.1. Scoring function. Example 1:

In [4]:
from pgmpy.estimators import BDeuScore, K2Score, BicScore

bdeu = BDeuScore(data, equivalent_sample_size=5)
k2 = K2Score(data)
bic = BicScore(data)

model1 = BayesianModel([('fruit', 'tasty'), ('size', 'tasty')]) # fruit -> tasty <- size
model2 = BayesianModel([('tasty', 'fruit'), ('tasty', 'size')]) # fruit <- tasty -> size

print(bdeu.score(model1))
print(k2.score(model1))
print(bic.score(model1))

print(bdeu.score(model2))
print(k2.score(model2))
print(bic.score(model2))


-30.12792467904587
-30.3772093643128
-32.859257093436106
-29.99714276768256
-30.551081866620226
-32.45409104969264


## 3.2. Scoring function. Example 2

In [5]:
# create random data sample with 3 variables, where Z is dependent on X, Y:
data = pd.DataFrame(np.random.randint(0, 4, size=(5000, 2)), columns=list('XY'))
data['Z'] = data['X'] + data['Y']

bdeu = BDeuScore(data, equivalent_sample_size=5)
k2 = K2Score(data)
bic = BicScore(data)

model1 = BayesianModel([('X', 'Z'), ('Y', 'Z')])  # X -> Z <- Y
model2 = BayesianModel([('X', 'Z'), ('X', 'Y')])  # Y <- X -> Z


print(bdeu.score(model1))
print(k2.score(model1))
print(bic.score(model1))

print(bdeu.score(model2))
print(k2.score(model2))
print(bic.score(model2))

-13936.48935953548
-14327.268987584197
-14292.528615686488
-20901.342569405366
-20928.175797309992
-20945.389990459298


## 3.3. One last example

Example with HillClimbSearch and BicScore.

In [6]:
from pgmpy.estimators import HillClimbSearch

# create some data with dependencies
data = pd.DataFrame(np.random.randint(0, 3, size=(2500, 8)), columns=list('ABCDEFGH'))
data['A'] += data['B'] + data['C']
data['H'] = data['G'] - data['A']

hc = HillClimbSearch(data)
best_model = hc.estimate(scoring_method=BicScore(data))
print(best_model.edges())

  0%|          | 0/1000000 [00:00<?, ?it/s]

[('A', 'H'), ('B', 'A'), ('C', 'A'), ('G', 'H')]


## 3.4. Questions
Based on the previous code provide an answer the following questions:

1. Analyse the results from 3.1 and 3.2
2. Analyse the structure of the obtained network.


1(3.1).

No modelo 1 fruit e size são independentes entre si e no modelo 2 fruit e size são dependentes entre si. Com os resultados, podemos concluir que dependendo do score, podemos obter melhores resultados com o modelo 1 ou com o modelo 2, com BICscore obtemos melhores resultados no modelo 1 e com BDeuscore e K2score obtemos melhores resultados no modelo 2.

1(3.2).

Aqui, o modelo 1 o X e o Y são independentes entre si e no modelo 2 X e Y são dependentes entre si. Obtemos valores maiores (negativamente) visto que temos um dataset maior. Em todos os scorings o modelo 1 obteve uma maior pontuação do que o modelo 2.

2.

No modelo 1 o size depende de tasty e este depende de fruit, enquanto que no modelo 2 tasty depende de fruit e de size e este depende de fruit.


### 4. Chalenge: All together now


1. Consider the file cancer.bif with the cancer disease BN already used.
2. Load the network and use the method *inference.likelihood_weighted_sample* to sample a minimumm of 5000 examples.
3. Calculate the CPDs and compare with the original ones.
4. Apply diferent strategies (scores and parameters) to determine a BN structure. Compare with the original.








In [7]:
from pgmpy.readwrite import BIFReader

bif = BIFReader("/content/cancer.bif")
model = bif.get_model()
edges = bif.get_edges()
og_cpds = model.get_cpds()

In [8]:
print("CPDs Originais")
for cpd in og_cpds:
  print(cpd)

CPDs Originais
+---------------+----------------+----------------+-----------------+-----------------+
| Pollution     | Pollution(low) | Pollution(low) | Pollution(high) | Pollution(high) |
+---------------+----------------+----------------+-----------------+-----------------+
| Smoker        | Smoker(True)   | Smoker(False)  | Smoker(True)    | Smoker(False)   |
+---------------+----------------+----------------+-----------------+-----------------+
| Cancer(True)  | 0.03           | 0.001          | 0.05            | 0.02            |
+---------------+----------------+----------------+-----------------+-----------------+
| Cancer(False) | 0.97           | 0.999          | 0.95            | 0.98            |
+---------------+----------------+----------------+-----------------+-----------------+
+-----------------+--------------+---------------+
| Cancer          | Cancer(True) | Cancer(False) |
+-----------------+--------------+---------------+
| Dyspnoea(True)  | 0.65         | 0.3  

In [9]:
from pgmpy.sampling import BayesianModelSampling

inference = BayesianModelSampling(model)
samples = inference.likelihood_weighted_sample(size=5000)
samples.to_csv("cancer_samples.csv")

  0%|          | 0/5 [00:00<?, ?it/s]

In [10]:
# Use a MLE estimator to obtain the CPDs tables
print("CPDs depois do uso do método inference.likelihood_weighted_sample")
model.fit(samples, estimator=MaximumLikelihoodEstimator)
for cpd in model.get_cpds():
    print(cpd)

CPDs depois do uso do método inference.likelihood_weighted_sample
+---------------+----------------------+-----+----------------------+
| Pollution     | Pollution(high)      | ... | Pollution(low)       |
+---------------+----------------------+-----+----------------------+
| Smoker        | Smoker(False)        | ... | Smoker(True)         |
+---------------+----------------------+-----+----------------------+
| Cancer(False) | 0.9749303621169917   | ... | 0.9702159344750558   |
+---------------+----------------------+-----+----------------------+
| Cancer(True)  | 0.025069637883008356 | ... | 0.029784065524944156 |
+---------------+----------------------+-----+----------------------+
+-----------------+---------------------+--------------------+
| Cancer          | Cancer(False)       | Cancer(True)       |
+-----------------+---------------------+--------------------+
| Dyspnoea(False) | 0.6938031591737546  | 0.3225806451612903 |
+-----------------+---------------------+-----------

In [11]:
# Use a Bayesian estimator with BDeu prior_type to obtain the CPDs tables
model.fit(samples, estimator=BayesianEstimator, prior_type="BDeu")
for cpd in model.get_cpds():
    print(cpd)

+---------------+----------------------+-----+----------------------+
| Pollution     | Pollution(high)      | ... | Pollution(low)       |
+---------------+----------------------+-----+----------------------+
| Smoker        | Smoker(False)        | ... | Smoker(True)         |
+---------------+----------------------+-----+----------------------+
| Cancer(False) | 0.9732824427480916   | ... | 0.969778687000186    |
+---------------+----------------------+-----+----------------------+
| Cancer(True)  | 0.026717557251908396 | ... | 0.030221312999814023 |
+---------------+----------------------+-----+----------------------+
+-----------------+--------------------+---------------------+
| Cancer          | Cancer(False)      | Cancer(True)        |
+-----------------+--------------------+---------------------+
| Dyspnoea(False) | 0.6937050905778768 | 0.32945736434108525 |
+-----------------+--------------------+---------------------+
| Dyspnoea(True)  | 0.3062949094221233 | 0.670542635658

In [12]:
# Use a Bayesian estimator with K2 prior_type to obtain the CPDs tables
model.fit(samples, estimator=BayesianEstimator, prior_type="K2")
for cpd in model.get_cpds():
    print(cpd)

+---------------+----------------------+-----+----------------------+
| Pollution     | Pollution(high)      | ... | Pollution(low)       |
+---------------+----------------------+-----+----------------------+
| Smoker        | Smoker(False)        | ... | Smoker(True)         |
+---------------+----------------------+-----+----------------------+
| Cancer(False) | 0.9722991689750693   | ... | 0.9695167286245353   |
+---------------+----------------------+-----+----------------------+
| Cancer(True)  | 0.027700831024930747 | ... | 0.030483271375464683 |
+---------------+----------------------+-----+----------------------+
+-----------------+--------------------+--------------+
| Cancer          | Cancer(False)      | Cancer(True) |
+-----------------+--------------------+--------------+
| Dyspnoea(False) | 0.6937246963562753 | 0.328125     |
+-----------------+--------------------+--------------+
| Dyspnoea(True)  | 0.3062753036437247 | 0.671875     |
+-----------------+---------------

In [13]:
bdeu = BDeuScore(samples, equivalent_sample_size=5)
k2 = K2Score(samples)
bic = BicScore(samples)

print(bdeu.score(model))
print(k2.score(model))
print(bic.score(model))

-10568.352920561341
-10570.81652649889
-10574.292021528956


In [14]:
hc = HillClimbSearch(samples)

best_model_Bdeu = hc.estimate(scoring_method=BDeuScore(samples))
print("### BdeuScore ###")
print(best_model_Bdeu.edges())
print(bic.score(best_model_Bdeu))

best_model_K2 = hc.estimate(scoring_method=K2Score(samples))
print("### K2Score ###")
print(best_model_K2.edges())
print(bic.score(best_model_K2))

best_model_Bic = hc.estimate(scoring_method=BicScore(samples))
print("### BicScore ###")
print(best_model_Bic.edges())
print(bic.score(best_model_Bic))

  0%|          | 0/1000000 [00:00<?, ?it/s]

### BdeuScore ###
[('Pollution', 'Cancer'), ('Smoker', 'Cancer'), ('Cancer', 'Xray'), ('Cancer', 'Dyspnoea')]
-10574.292021528956


  0%|          | 0/1000000 [00:00<?, ?it/s]

### K2Score ###
[('Pollution', 'Smoker'), ('Cancer', 'Xray'), ('Cancer', 'Smoker'), ('Cancer', 'Dyspnoea'), ('Cancer', 'Pollution')]
-10578.550429982204


  0%|          | 0/1000000 [00:00<?, ?it/s]

### BicScore ###
[('Cancer', 'Xray'), ('Cancer', 'Smoker'), ('Cancer', 'Dyspnoea'), ('Cancer', 'Pollution')]
-10576.481645286594
